Módulo 3 - Regressão
========================================================

Professor: Marcos Cesar Gritti  
Email: cesargritti@gmail.com

Antes de começar:
 - Crie uma cópia deste notebook, e o renomeie para "aula_1_modulo_3_{seu_nome}";
 - Caso seu ambiente Anaconda não possua uma das dependências necessárias para a execução do código contigo neste notebook, abra uma célula e execute o comando: ```!pip install -r ../requirements.txt```

Neste módulo vamos aprender sobre:
 1. **Seleção/organização de features (variáveis de entrada)**;
 1. **Particionamento de dados (Conjunto de treinamento e de validação)**;
 2. **Regressão usando Regressão Linear Multipla e Random Forest**;
 
A aula de hoje será um exercício de avaliação, que deverá ser entregue no **Google Classroom**.

Carregando os dados do exercício de hoje
===================================

Os dados que utilizaremos na aula de hoje estão salvos no arquivo `dados_tratados.csv`. <br></br>

**Exercício**: Carregue-os em memória em uma variável chamada `dados`, na célula abaixo, utilizando a biblioteca ``pandas``.

In [112]:
import pandas as pd

dados = pd.read_csv('./dados_tratados.csv', sep=',', header='infer')

dados.sample(n=5)

,localidade,tipo,loc_x,loc_y,mercado_mais_proximo,farmacia_mais_proxima,escola_mais_proxima,num_penit_4km,num_penit_500m,idade_imovel,area,preco
350,Localidade 3,Tipo 1,0.818434,0.500433,2208.366340,1748.289375,1603.416750,0,0,46,133.0,552700
1553,Localidade 3,Tipo 1,0.034274,0.307058,4123.744797,3148.933856,1422.941686,0,0,48,177.0,523200
1562,Localidade 1,Tipo 1,0.025341,0.761213,4751.388191,253.814331,1812.971200,0,0,18,119.0,739500
961,Localidade 4,Tipo 1,0.772359,0.722433,6389.779663,2306.225617,2478.407798,1,0,15,105.0,529700
1718,Localidade 2,Tipo 1,0.245855,0.614833,3538.999314,3105.689359,2745.877183,1,0,6,192.0,186100


1 - Seleção/organização de features (variáveis de entrada)
===============================================

São poucos os algorítmos de **Machine Learning** que conseguem lidar com variáveis categóricas sem um devido pré-processamento. Por esse motivo, é importante codificá-las de forma adequada, para que seja possível comparar técnicas resultados de técnicas distintas, mesmo que uma delas não trate valores categóricos internamente.

Os métodos mais utilizados para codificação de variáveis categóricas são:
 - **One hot encoding**: Criar uma coluna para cada possível valor de saída de sua variável categórica. Quando a sua amostra pertence à **classe** representada por uma determinada coluna, atribuimos o valor 1 (essa amostra percente a esta classe), caso contrário, a célula desta coluna recebe o valor 0 (essa amostra não pertence a esta classe);
 - **Variáveis Dummy**: Similar a codificação **One hot encoding**, porém, omite uma das possíveis colunas de saída. Por exemplo, se nossa variável categórica "*Tipo*" pode assumir os valores A, B e C, criamos apenas duas colunas, denominadas *Tipo_A* e *Tipo_B*. Note que, na ausência das duas (valor = 0), sabemos que estamos nos referindo a uma amostra do *Tipo_C*.
 
**Exercício**: Codifique as variáveis categóricas (**localidade** e **tipo**) do conjunto de dados, e salve o
resultado em um variável chamada ``dataset``, contendo apenas as **features** (variáveis de entrada) e o **target** (valor do imóvel, variável **preco**) que utilizaremos na análise de regressão.

**Dicas**:
 - Ao menos que você tenha projetado uma camada de **Imputação de Dados** para o seu problema, lembre-se de remover dados nulos do seu conjunto antes de organizar as suas **features**;
 - Procure pela documentação da função ``get_dummies`` da biblioteca ``pandas``;

In [113]:
#### 1 - Tratamento dos Dados Nulos
method = 'input'
#method = 'drop'

## 1.1 Removendo dados nulos  OU   Inputação dos valores com base na média
if method == 'input':
    dados.fillna(dados.mean(), inplace=True)
else:
    dados.dropna(inplace=True)

dados.isnull().any(axis=1).value_counts()

False    1833
dtype: int64

In [114]:
#### 2 - Codificação das variaveis categoricas 

## 2.1 Encontrar as variaveis categoricas e numericas
numVar = dados.select_dtypes(include=['number']).columns
catVar = dados.select_dtypes(include=['object']).columns

In [115]:
## 2.2 Codigifcação (one hot encoding)
loc = pd.get_dummies(dados['localidade'])
tip = pd.get_dummies(dados['tipo'])

dataset = pd.concat([dados, loc, tip], axis=1)
dataset.sample(n=5)

,localidade,tipo,loc_x,loc_y,mercado_mais_proximo,farmacia_mais_proxima,escola_mais_proxima,num_penit_4km,num_penit_500m,idade_imovel,area,preco,Localidade 1,Localidade 2,Localidade 3,Localidade 4,Tipo 1,Tipo 2
1109,Localidade 3,Tipo 1,0.707061,0.055384,2003.096088,2884.771024,2714.020152,0,0,38,208.0,674200,0,0,1,0,1,0
597,Localidade 4,Tipo 1,0.790472,0.310253,2864.225499,3842.073936,618.756240,0,0,7,137.0,711200,0,0,0,1,1,0
824,Localidade 2,Tipo 2,0.059108,0.942754,6855.569267,4960.998732,5270.339193,1,0,9,95.0,295400,0,1,0,0,0,1
1036,Localidade 2,Tipo 1,0.641013,0.660999,2873.415017,1561.338907,1283.050498,1,0,2,153.0,204300,0,1,0,0,1,0
776,Localidade 2,Tipo 1,0.352486,0.175661,2816.375170,5312.909867,4572.995017,0,0,7,84.0,203500,0,1,0,0,1,0


In [116]:

## 2.3 Elencação das Features e Target
features = list(numVar)
features.remove('preco')
target = ['preco']
dataset = dataset[features + target]
print(f"""
    FEATURE:  { features }
    TARGET :  { target   }
""")


    FEATURE:  ['loc_x', 'loc_y', 'mercado_mais_proximo', 'farmacia_mais_proxima', 'escola_mais_proxima', 'num_penit_4km', 'num_penit_500m', 'idade_imovel', 'area']
    TARGET :  ['preco']



In [117]:
## 2.4 Normalizar os dados entre 0 e 1
dataset_norm = dataset.apply(lambda x:(x-x.min()) / (x.max()-x.min()))
dataset_norm.sample(n=5)

,loc_x,loc_y,mercado_mais_proximo,farmacia_mais_proxima,escola_mais_proxima,num_penit_4km,num_penit_500m,idade_imovel,area,preco
1584,0.962107,0.267656,0.211601,0.089340,0.401612,0.0,0.0,0.10,0.112821,0.065279
1470,0.671431,0.005187,0.230628,0.327738,0.336378,0.0,0.0,0.34,0.492308,0.536308
1557,0.044420,0.048037,0.451886,0.851825,0.207884,0.5,0.0,0.28,0.312821,0.408295
1024,0.100678,0.566255,0.316107,0.570071,0.577934,0.5,0.0,0.06,0.605128,0.048959
673,0.470423,0.055555,0.126203,0.190994,0.516756,0.0,0.0,0.16,0.292308,0.156985


In [118]:
## 2.5 Criacao dos conjuntos X e y
#X = dataset_norm[features]
X = dataset_norm[features].copy()
y = dataset_norm[target].copy()

2 - Particionamento de dados
========================

Particionar dados significa dividí-los em conjuntos de amostras para fins distintos. Em **Data Mining** e **Machine Learning**, é importante segregar os dados em um conjunto de **treinamento** e um conjunto de **teste**.

**Por que?**

Precisamos separar alguns dados para teste para que possamos verificar a qualidade da solução de forma justa, ou seja, em um conjunto de dados não visto previamente pelo algorítmo no momento de treinamento do modelo. Essa técnica recebe o nome de **Validação Cruzada** (*Cross Validation*). Existem várias formas de aplicar a **Validação Cruzada**:
 - Segregação em **Treinamento** e **Teste**;
 - Segregação em **Treinamento** e **Teste** com estratificação;
 - Segregação em **Treinamento**, **Teste** e validação;
 - KFolds (Vários conjuntos de **Treinamento** e **Teste** para otimização de hiperparâmetros);
 - Entre outras técnicas utilizadas, por exemplo, em dados que dependem do tempo (Séries temporais);
 
Neste exercício, vamos segregar os dados em: um conjunto de **treinamento**; e um conjunto de **teste**.

Particione os dados de entrada $X$ e saída $y$ da etapa anterior para que possamos dar continuidade na
análise de regressão. Para isso, crie/utiliza as células que estão entre essa seção (2) e a próxima.
Os conjuntos de treinamento e de teste devem ser salvos, respectivamente, em variáveis chamadas
`X_train`, `y_train`, `X_test` e `y_test`.

**Dica**: Procure pela documentação da função `train_test_split`, pertencente ao pacote `model_selection` da biblioteca `sklearn`. 

In [119]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

3 - Regressão usando Regressão Linear Multipla e Random Forest
======================================================

Agora que temos dados particionados, podemos iniciar a modelagem.

**Exercício**: Crie dois modelos de regressão (Regressão Linear Multipla e Random Forest) utilizando os
dados de **treinamento**. Apresente os resultados obtidos (para cada modelo) no conjunto de **test** para o professor ulizando a métrica MAPE (*Mean Absolute Percentage Error*), descrita pela seguinte fórmula:

\begin{equation}
    \text{MAPE} = \frac{1}{N} \sum\limits_{i=1}^N \left| \frac{y_i - \hat{y}_i}{y_i} \right|
\end{equation}

Por fim, crie uma cópia do **dataframe** inicial (``dados``), e adicione a ele as seguintes colunas:
 - pred_lm: A previsão do valor do imóvel (Modelo Linear);
 - erro_lm: O erro de nossa predição (Modelo Linear);
 - ape_lm: O erro percentual de nossa predição (Modelo Linear);
 - pred_rf: A previsão do valor do imóvel (Random Forest);
 - erro_rf: O erro de nossa predição (Random Forest);
 - ape_rf: O erro percentual de nossa predição (Random Forest);
 
**Dicas**: Procure pelas seguintes documentações da biblioteca `sklearn`:
- classe `LinearRegression` percentente ao pacote `linear_model`;
- classe `RandomForestRegression` pertencente ao pacote `ensemble`;

In [120]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import __version__

print(__version__)

#MAPE esta disponivel somente na versão 0.24
def mape(y, yp):
    return mean_absolute_percentage_error(y, yp)

def mse(y, yp):
    return mean_squared_error(y, yp)

def r2(y, yp):
    return r2_score(y, yp)

0.24.2


In [121]:
from sklearn.linear_model import LinearRegression

lm_model = LinearRegression(fit_intercept=True)
lm_model.fit(X_train, y_train)

lm_result = pd.concat([X_test, y_test], axis=1)

lm_result['pred_lm'] = lm_model.predict(X_test)
lm_result['erro_lm'] = lm_result.preco - lm_result.pred_lm
lm_result['ape_lm'] = (lm_result.erro_lm.abs() / lm_result.preco) * 100

In [122]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

rf_result = pd.concat([X_test, y_test], axis=1)

rf_result['pred_rf'] = rf_model.predict(X_test)
rf_result['erro_rf'] = rf_result.preco - rf_result.pred_rf
rf_result['ape_rf'] = (rf_result.erro_rf.abs() / rf_result.preco) * 100

C:\Users\melch\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [123]:
print(f"""
    Linear Regression Model Results 
    {lm_result['ape_lm'].describe()}
MAPE:\t {mape(lm_result['preco'], lm_result['pred_lm']) * 100 :.6f} %
MSE :\t  {mse(lm_result['preco'], lm_result['pred_lm']) :.6f}
R2  :\t {r2(lm_result['preco'], lm_result['pred_lm']) *100 :.6f} %
    
    Random Forest Regression Model Results
    {rf_result['ape_rf'].describe()}
MAPE:\t {mape(rf_result['preco'], rf_result['pred_rf']) * 100 :.6f} %
MSE :\t  {mse(rf_result['preco'], rf_result['pred_rf']) :.6f}
R2  :\t {r2(rf_result['preco'], rf_result['pred_rf']) * 100 :.6f} %
""")


    Linear Regression Model Results 
    count    459.000000
mean      96.268491
std      126.201484
min        0.032295
25%       21.595248
50%       45.182315
75%      114.270135
max      799.718874
Name: ape_lm, dtype: float64
MAPE:	 96.268491 %
MSE :	  0.025476
R2  :	 20.229988 %
    
    Random Forest Regression Model Results
    count     459.000000
mean       58.134755
std        90.445314
min         0.105954
25%        12.931794
50%        31.623870
75%        67.975580
max      1086.710177
Name: ape_rf, dtype: float64
MAPE:	 58.134755 %
MSE :	  0.017066
R2  :	 46.563730 %

